# Control moment gyroscope

Import modules and configure the notebook.

In [2]:
# This module is part of the python standard library
import time

# These modules are part of other existing libraries
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt
import scipy

# This is my own script (it is an interface to the pybullet simulator)
import ae353_cmg

Create an instance of the `Simulator` class, which is an interface to the [pybullet](http://pybullet.org) simulation engine.

There are several optional arguments:

* `display` (`True` or `False`) is whether or not to show the simulation window;
* `damping` (a non-negative floating-point number, default `0.`) is the coefficient of viscous friction at each joint;
* `load_mass` (a non-negative floating-point number, default `1.`) is the mass of the sphere attached to the platform;
* `roll` (an angle in radians, default `np.pi / 2`) is the roll angle of the ground.

In [3]:
simulator = ae353_cmg.Simulator(
    display=False,
    damping=0.,
    load_mass=1.,
    roll=(np.pi / 2),
)

Choose a default camera view (could also be `simulator.camera_topview()`).

In [4]:
simulator.camera_sideview()

Define a controller that maps sensor measurements to actuator commands. By default, it chooses to apply zero torque to the gimbal. Modify the `run` function to make the controller behave differently. This function will be called 100 times per second (i.e., at 100 Hz) by the simulator.

In [5]:
Amatrix= np.array([[0,0,1,0],[0,0,0,1],[-4.349368,0,0,1.108402],[0,0,-454.5455,0]]) # 
Bmatrix=np.array([[0],[0],[0],[90.90909]])

In [6]:

        # j values + constants
        j1z = 0.5 # kg * m^2
        j2x = 0.001
        j2z = 0.001 # kg * m^2
        j3x = 0.01
        j3z = 0.01
        j3y = 0.01 # kg * m^2
        
        m = 1.0 # kg
        r = 2.0 # m
        g = 9.81 # m/s^2
        v_rotor = 500 # rpm
        
        # a values
        a1 = -j3y + 2*j3z
        a2 = 2*j3y
        a3 = -2*g*m*r
        a4 = 2*j1z + 2*j2z + 2*m*r**2
        a5 = 2*j3z
        a6 = (j3y - j3z) / (2 * (j2x + j3x))
        a7 = -j3y / (j2x + j3x)
        a8 = 1 / (j2x + j3x)
        
        # do the math stuff
        
        q1, q2, v1, v2, tau = sym.symbols('q1, q2, v1, v2, tau')
        f = sym.Matrix([v1, v2, (a1*sym.sin(2*q2)*v1*v2 + a2*sym.cos(q2)*v2*v_rotor + a3*sym.sin(q1)) / (a4 + a5*sym.cos(q2)**2), a6*sym.sin(2*q2)*v1**2 + a7*sym.cos(q2)*v1*v_rotor + a8*tau])
        f_num = sym.lambdify([q1, q2, v1, v2, tau], f)
        
        # equilibrium points
        q_1 = 0
        q_2 = 0
        v_1 = 0
        v_2 = 0
        tau_ = 0
        
        f_num(q_1, q_2, v_1, v_2, tau)
        
        A_num = sym.lambdify((q1, q2, v1, v2, tau), f.jacobian([q1, q2, v1, v2]))
        A = A_num(q_1, q_2, v_1, v_2, tau_)
        A = A.astype(float)

        B_num = sym.lambdify((q1, q2, v1, v2, tau), f.jacobian([tau]), 'numpy')
        B = B_num(q_1, q_2, v_1, v_2, tau_)
        B = B.astype(float)
        
        print(A)
        print(B)

[[   0.            0.            1.            0.        ]
 [   0.            0.            0.            1.        ]
 [  -4.34936821    0.            0.            1.10840168]
 [   0.            0.         -454.54545455    0.        ]]
[[ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [90.90909091]]


In [7]:
f

Matrix([
[                                                                                v1],
[                                                                                v2],
[(0.01*v1*v2*sin(2*q2) + 10.0*v2*cos(q2) - 39.24*sin(q1))/(0.02*cos(q2)**2 + 9.002)],
[                                90.9090909090909*tau - 454.545454545455*v1*cos(q2)]])

## Randomization of K value

In [ ]:
import random

check = True
k = np.array([[0, 0, 0, 0]])
while check:
    a = random.uniform(-15,15)
    b = random.uniform(-15,15)
    c = random.uniform(-15,15)
    d = random.uniform(-15,15)
    k = np.array([[a, b, c, d]])
    vals = np.linalg.eigvals(Amatrix-Bmatrix@k)
    if (vals[0] < 0 and vals[1] < 0 and vals[2] < 0 and vals[3] < 0):
        check = False
print(k)
print(np.linalg.eigvals(Amatrix-Bmatrix@k))


## Recording Good K Values

In [ ]:
# another potential matrix

another = np.array([[ 5.58237073,  3.54115713, 12.98960478,  3.62212887]])
yetanother = np.array([[-11.25241229,   8.06963557,   4.9863024,    8.71055433]])

thistheone = np.array([[11.52962522,  5.04217526, -2.29439909,  1.62595883]])
thistheone

## This is the good matrix, [[55.2492422  29.34571649 54.25320811 96.71134327]]

In [ ]:
j = np.array([[55.2492422, 29.34571649, 54.25320811, 96.71134327]])
print(j)

In [ ]:
class Controller:
    def __init__(self):
        pass
    
    def reset(self):
        pass
    
    def run(
            self,
            t,
            platform_angle,
            platform_velocity,
            gimbal_angle,
            gimbal_velocity,
        ):
        
        # given variables
        

#         print(A.tolist())
#         print(B.tolist())
        
        qe = np.pi
        x = np.array([[platform_angle-qe], [gimbal_angle], [platform_velocity], [gimbal_velocity]])
        u = -thistheone@x
        
        gimbal_torque = float(u)
        
        return gimbal_torque

Create an instance of the controller.

In [ ]:
controller = Controller()

Reset the simulator with given initial conditions. A separate PID controller, which you don't have to implement, will keep the rotor velocity constant.

## Max Initial Conditions

Platform Angle: $1.64$ rad for $0 \leq \theta \leq \pi$ rad, $4.64$ for $\pi \leq \theta \leq 2\pi$ rad

Platform Velocity: $-3.47 to 3.46$

Gimbal Angle: any $\theta$ for $0 \leq \theta \leq 2\pi$ rad

Gimbal Velocity: $-119.50$ to $125.62$  $\frac{rad}{s}$

In [ ]:
simulator.reset(
    platform_angle=0,
    platform_velocity=0,
    gimbal_angle=0,
    gimbal_velocity=0,
    rotor_velocity=500.,
)

Reset the controller.

In [ ]:
controller.reset()

Run the simulator.

In [ ]:
data = simulator.run(
    controller,           # <-- required (an instance of your Controller class)
    max_time=60,         # <-- optional (how long you want to run the simulation in seconds)
    data_filename=None,   # <-- optional (name of file to which you want data saved, e.g., 'my_data.json')
    video_filename=None,  # <-- optional (name of file to which you want video saved, e.g., 'my_video.mov')
)

Get, show, and save a snapshot of the simulation after the last time step.

In [ ]:
# Get snapshot as height x width x 4 numpy array of RGBA values
rgba = simulator.snapshot()

# Display snapshot
plt.figure(figsize=(8, 8))
plt.imshow(rgba)

# Save snapshot
plt.imsave('my_snapshot.png', rgba)

Plot results.

In [ ]:
# Create a figure with three subplots, all of which share the same x-axis
fig, (ax_platform, ax_gimbal, ax_rotor, ax_torque) = plt.subplots(4, 1, figsize=(8, 8), sharex=True)

# Plot platform angle and velocity
ax_platform.plot(
    data['t'], data['platform_angle'],
    label='platform angle (rad)', linewidth=4,
)
ax_platform.plot(
    data['t'], data['platform_velocity'],
    '--', label='platform velocity (rad / s)', linewidth=4,
)
ax_platform.grid()
ax_platform.legend(fontsize=16)
ax_platform.tick_params(labelsize=14)

# Plot gimbal angle and velocity
ax_gimbal.plot(
    data['t'], data['gimbal_angle'],
    label='gimbal angle (rad)', linewidth=4,
)
ax_gimbal.plot(
    data['t'], data['gimbal_velocity'],
    '--', label='gimbal velocity (rad / s)', linewidth=4,
)
ax_gimbal.grid()
ax_gimbal.legend(fontsize=16)
ax_gimbal.tick_params(labelsize=14)

# Plot rotor velocity
ax_rotor.plot(
    data['t'], data['rotor_velocity'],
    '--', label='rotor velocity (rad / s)', linewidth=4,
)
ax_rotor.grid()
ax_rotor.legend(fontsize=16)
ax_rotor.tick_params(labelsize=14)
ax_rotor.set_ylim(simulator.rotor_velocity - 10., simulator.rotor_velocity + 10.)

# Plot torques
ax_torque.plot(
    data['t'], data['gimbal_torque_command'],
    label='gimbal torque command (N-m)', linewidth=4,
)
ax_torque.plot(
    data['t'], data['gimbal_torque'],
    '--', label='gimbal torque (N-m)', linewidth=4,
)
ax_torque.plot(
    data['t'], data['rotor_torque'],
    '--', label='rotor torque (N-m)', linewidth=4,
)
ax_torque.plot(
    data['t'], np.ones_like(data['t']) * simulator.tau_max,
    ':', label='max torque', linewidth=4, color='C4', zorder=0,
)
ax_torque.plot(
    data['t'], -np.ones_like(data['t']) * simulator.tau_max,
    ':', linewidth=4, color='C4', zorder=0,
)
ax_torque.grid()
ax_torque.legend(fontsize=16)
ax_torque.tick_params(labelsize=14)

# Set x-axis properties (only need to do this on the last
# subplot since all subplots share the same x-axis)
ax_torque.set_xlabel('time (s)', fontsize=20)
ax_torque.set_xlim([data['t'][0], data['t'][-1]])
ax_torque.set_ylim(-1.2 * simulator.tau_max, 1.2 * simulator.tau_max)

# Make the arrangement of subplots look nice
fig.tight_layout()

Save the plots to a file.

In [ ]:
fig.savefig('my_figure.png', facecolor='white', transparent=False)

## Defined Error Value

In [ ]:
con_error = 1*10**-10
con_error

## Platform Angle Data

In [ ]:
maxpa = data['platform_angle'][-500:].max()
minpa = data['platform_angle'][-500:].min()
print(maxpa, minpa)
e_range_pa = maxpa + minpa
e_range_pa

In [ ]:
if e_range_pa < con_error:
    print("Platform Angle converges, as error is within acceptable range")
else: print("Does not converge :((")

## Platform Velocity

In [68]:
maxpv = data['platform_velocity'][-500:].max()
minpv = data['platform_velocity'][-500:].min()
print(maxpv, minpv)
e_range_pv = maxpv + minpv
e_range_pv

4.264847186263729e-14 -3.7768666723441743e-14


4.879805139195546e-15

In [69]:
if e_range_pv < con_error:
    print("Platform Velocity converges, as error is within acceptable range")
else: print("Does not converge :((")

Platform Velocity converges, as error is within acceptable range


## Gimbal Angle

In [70]:
maxga = data['gimbal_angle'][-500:].max()
minga = data['gimbal_angle'][-500:].min()
print(maxga, minga)
e_range_ga = maxga - minga
e_range_ga

7.183682443000916 7.183682443000916


0.0

In [71]:
if e_range_ga < con_error:
    print("Gimbal Angle converges, as error is within acceptable range")
else: print("Does not converge :((")

Gimbal Angle converges, as error is within acceptable range


## Gimbal Velocity 

In [72]:
maxgv = data['gimbal_velocity'][-500:].max()
mingv = data['gimbal_velocity'][-500:].min()
print(maxgv, mingv)
e_range_gv = maxgv + mingv
e_range_gv

1.0018537969216004e-13 -8.709880189080775e-14


1.3086577801352299e-14

In [73]:
if e_range_gv < con_error:
    print("Gimbal Velocity converges, as error is within acceptable range")
else: print("Does not converge :((")

Gimbal Velocity converges, as error is within acceptable range
